Пытаюсь найти нормальный алгоритм для выделения ключевых моментов

# 0

In [2]:
%load_ext autoreload
%autoreload 2
%matplotlib inline 

import os
import sys
import json
from pathlib import Path
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from tqdm import tqdm
from collections import Counter, defaultdict


selected_videos = [25, 5, 14, 17, 27]
suf = selected_videos[2]
path_mp4 = Path('/storage/arkady/Glinka/music-transformer/video_conditioning') \
    / 'content' / 'video_data' / f'video_ad{suf}.mp4'
assert os.path.exists(path_mp4)

# kvm3

In [29]:
from video_utils.kvm import extract_kvm3, extract_kvm2, get_video_duration, search_thresholds_for_kvm3


path_videos = Path('/storage/arkady/Glinka/music-transformer/video_conditioning') \
    / 'content' / 'video_data'

for path_video in sorted(os.listdir(path_videos)):
    if not path_video.endswith('mp4'):
        continue

    print(path_video)
    path_mp4 = path_videos / path_video 
    
    kvm3, fps3, *kvm4 = extract_kvm3(path_mp4, threshold=90.0)
    kvm3 = np.array(kvm3) / fps3

    kvm4 = kvm4[0]
    kvm4 = [k[0].get_seconds() for k in kvm4] + [kvm4[-1][1].get_seconds()]

    duration, *a = get_video_duration(path_mp4)
    print(duration)

    break
extract_kvm3(path_mp4, threshold=90.0)

VideoManager is deprecated and will be removed.


video_ad1.mp4


VideoManager is deprecated and will be removed.


100.4003


([814, 1688, 1777, 1849, 2157, 2715],
 29.97002997002997,
 [(00:00:00.000 [frame=0, fps=29.970], 00:00:54.354 [frame=1629, fps=29.970]),
  (00:00:54.354 [frame=1629, fps=29.970],
   00:00:58.325 [frame=1748, fps=29.970]),
  (00:00:58.325 [frame=1748, fps=29.970],
   00:01:00.260 [frame=1806, fps=29.970]),
  (00:01:00.260 [frame=1806, fps=29.970],
   00:01:03.163 [frame=1893, fps=29.970]),
  (00:01:03.163 [frame=1893, fps=29.970],
   00:01:20.814 [frame=2422, fps=29.970]),
  (00:01:20.814 [frame=2422, fps=29.970],
   00:01:40.400 [frame=3009, fps=29.970])],
 3009)

https://colab.research.google.com/drive/19KdwIIO0qxppDj41qrhBkEJtOQxTJeN2#scrollTo=_DgJ_vLUyxfi

## kvm1 CNN for action recognition - Microsoft/cv

Привет, я посмотрел, там все в целом работает, просто он внутри функции в if не заходит и поэтому
None возвращается и все падает с эксепшеном. Я на рандомном видео поменял параметр averaging_size
на 1, тогда все заработало (по крайней мере он начал заходить в if и что-то считать)

Ну и там надо второй return делать из тех что у тебя написаны

Дополнения в коде /content/computervision-recipes/utils_cv/action_recognition/model.py в predict_frames
```
# top5 = None
# ...
# return top5[0][0] if top5 and len(top5) else ''
# or return (top5[0][0], set([el[0] for el in top5])) if top5 and len(top5) else ('', set())
```

In [ ]:
!pip list | grep decord

In [ ]:
import cv2
import numpy as np
import decord
import torch
from collections import deque
from utils_cv.action_recognition.data import KINETICS
from utils_cv.action_recognition.dataset import get_transforms
from utils_cv.action_recognition.model import VideoLearner
from ipywidgets import HBox, HTML, Layout, VBox, Widget, Label
LABELS = KINETICS.class_names
NUM_FRAMES = 8
SCORE_THRESHOLD = 0.16

learner = VideoLearner(
    base_model="kinetics",
    sample_length=NUM_FRAMES,
	
)

transforms = get_transforms(train=False)

Loading r2plus1d_34_8_kinetics model


Using cache found in /home/arkady/.cache/torch/hub/moabitcoin_ig65m-pytorch_master


In [ ]:
def func(*args):
    return learner.predict_frames(args)

def VideoToMoments(path_video, SCORE_THRESHOLD=0.16) -> list:
    print(10)
    path_video = str(path_video)
    cap = cv2.VideoCapture(path_video)
    print(20) 
    FPS = int(cap.get(cv2.CAP_PROP_FPS))
    print(f'Video: {path_video}, FPS: {FPS}')

    video_reader = decord.VideoReader(path_video)
    window = deque(maxlen=NUM_FRAMES)
    scores_cache = deque()
    scores_sum = np.zeros(len(LABELS))    
    w_text = HTML(layout=Layout(padding="0 0 0 100px"))
    
    key_video_moments = []
    prev_frame_sec = 0
    prev_sets = []

    # # for cur_frame, frame in tqdm(enumerate(video_reader)):
    cur_frame = -1
    # if torch.cuda.is_available():
    #     device = torch.device('cuda')
    # else:
    #     device = torch.device('cpu')

    # print(device)
    # learner.model.to(device)
    # learner.model.eval()

    while True:
        print('.', end='')
        cur_frame += 1
        try:
            frame = video_reader.next()
            print('|', end='')
            frame = frame.asnumpy()
            window.append(frame)
            def update_println(println):
                    w_text.value = println

            if len(window) == NUM_FRAMES:
                # Check if CUDA is available


                print(2)
                action, set_a = func(
                    window,
                    scores_cache,
                    scores_sum,
                    None,
                    30,
                    SCORE_THRESHOLD,
                    LABELS,
                    LABELS,
                    get_transforms(train=False),
                    update_println,
                )
                print(3)

        #         cur_frame_sec = cur_frame // FPS + (cur_frame % FPS) / FPS
        #         if len(prev_sets) > FPS:
        #             prev_cumm_set = set().union(*prev_sets[-FPS:])
        #         else:
        #             prev_cumm_set = set().union(*prev_sets)

        #         if action and (cur_frame == 0 or (action not in prev_cumm_set and cur_frame_sec - prev_frame_sec > 0.5)):
        #             key_video_moments.append(float(f"{cur_frame_sec:.2f}"))
        #             prev_frame_sec = cur_frame_sec
        #             prev_sets.append(set_a)
        except:
            return key_video_moments

    return key_video_moments


key_moments = VideoToMoments(path_mp4)
key_moments

10
20
Video: /storage/arkady/Glinka/music-transformer/video_conditioning/content/video_data/video_ad14.mp4, FPS: 25
.|.|.|.|.|.|.|.|2
.|2
.|2
.|2
.|2
.|2
.|2
.|2
.|2
.|2
.|2
.|2
.|2
.|2
.|2
.|2
.|2
.|2
.|2
.|2
.|2
.|2
.|2
.|2
.|2
.|2
.|2
.|2
.|2
.|2
.|2
.|2
.|2
.|2
.|2
.|2
.|2
.|2
.|2
.|2
.|2
.|2
.|2
.|2
.|2
.|2
.|2
.|2
.|2
.|2
.|2
.|2
.|2
.|2
.|2
.|2
.|2
.|2
.|2
.|2
.|2
.|2
.|2
.|2
.|2
.|2
.|2
.|2
.|2
.|2
.|2
.|2
.|2
.|2
.|2
.|2
.|2
.|2
.|2
.|2
.|2
.|2
.|2
.|2
.|2
.|2
.|2
.|2
.|2
.|2
.|2
.|2
.|2
.|2
.|2
.|2
.|2
.|2
.|2
.|2
.|2
.|2
.|2
.|2
.|2
.|2
.|2
.|2
.|2
.|2
.|2
.|2
.|2
.|2
.|2
.|2
.|2
.|2
.|2
.|2
.|2
.|2
.|2
.|2
.|2
.|2
.|2
.|2
.|2
.|2
.|2
.|2
.|2
.|2
.|2
.|2
.|2
.|2
.|2
.|2
.|2
.|2
.|2
.|2
.|2
.|2
.|2
.|2
.|2
.|2
.|2
.|2
.|2
.|2
.|2
.|2
.|2
.|2
.|2
.|2
.|2
.|2
.|2
.|2
.|2
.|2
.|2
.|2
.|2
.|2
.|2
.|2
.|2
.|2
.|2
.|2
.|2
.|2
.|2
.|2
.|2
.|2
.|2
.|2
.|2
.|2
.|2
.|2
.|2
.|2
.|2
.|2
.|2
.|2
.|2
.|2
.|2
.|2
.|2
.|2
.|2
.|2
.|2
.|2
.|2
.|2
.|2
.|2
.|2
.|2
.|2
.|2
.|2
.|2
.|2
.|2
.|2
.|

[]

In [ ]:
key_moments

[]

## kvm2

In [ ]:
from video_utils.kvm import extract_kvm2

kvm2, fps2 = extract_kvm2(path_mp4, thres=0.6)

np.array(kvm2) / fps2

  5%|▌         | 68/1282 [00:00<00:03, 359.44it/s]

100%|██████████| 1282/1282 [00:02<00:00, 484.54it/s]


array([ 1.  ,  2.44,  4.36,  6.64,  8.44,  9.68, 12.08, 19.72, 26.28,
       32.24, 36.76, 39.24, 39.72, 40.84, 41.96, 43.  , 46.16])

# С сайта https://programmersought.com/article/79879511598/

не разбирался в этом

In [ ]:
# -*- coding: utf-8 -*-
"""
 Inter maximum method
Created on Tue Dec  4 16:48:57 2018
keyframes extract tool
this key frame extract algorithm is based on interframe difference.
The principle is very simple
First, we load the video and compute the interframe difference between each frames
Then, we can choose one of these three methods to extract keyframes, which are 
all based on the difference method:
    
1. use the difference order
    The first few frames with the largest average interframe difference 
    are considered to be key frames.
2. use the difference threshold
    The frames which the average interframe difference are large than the 
    threshold are considered to be key frames.
3. use local maximum
    The frames which the average interframe difference are local maximum are 
    considered to be key frames.
    It should be noted that smoothing the average difference value before 
    calculating the local maximum can effectively remove noise to avoid 
    repeated extraction of frames of similar scenes.
After a few experiment, the third method has a better key frame extraction effect.
The original code comes from the link below, I optimized the code to reduce 
unnecessary memory consumption.
https://blog.csdn.net/qq_21997625/article/details/81285096
@author: zyb_as
""" 
import cv2
import operator
import numpy as np
import matplotlib.pyplot as plt
import sys
from scipy.signal import argrelextrema

 
def smooth(x, window_len=13, window='hanning'):
    """smooth the data using a window with requested size.
    
    This method is based on the convolution of a scaled window with the signal.
    The signal is prepared by introducing reflected copies of the signal 
    (with the window size) in both ends so that transient parts are minimized
    in the begining and end part of the output signal.
    
    input:
        x: the input signal 
        window_len: the dimension of the smoothing window
        window: the type of window from 'flat', 'hanning', 'hamming', 'bartlett', 'blackman'
            flat window will produce a moving average smoothing.
    output:
        the smoothed signal
        
    example:
    import numpy as np    
    t = np.linspace(-2,2,0.1)
    x = np.sin(t)+np.random.randn(len(t))*0.1
    y = smooth(x)
    
    see also: 
    
    numpy.hanning, numpy.hamming, numpy.bartlett, numpy.blackman, numpy.convolve
    scipy.signal.lfilter
 
    TODO: the window parameter could be the window itself if an array instead of a string   
    """
    print(len(x), window_len)
    # if x.ndim != 1:
    #     raise ValueError, "smooth only accepts 1 dimension arrays."
    #
    # if x.size < window_len:
    #     raise ValueError, "Input vector needs to be bigger than window size."
    #
    # if window_len < 3:
    #     return x
    #
    # if not window in ['flat', 'hanning', 'hamming', 'bartlett', 'blackman']:
    #     raise ValueError, "Window is on of 'flat', 'hanning', 'hamming', 'bartlett', 'blackman'"
 
    s = np.r_[2 * x[0] - x[window_len:1:-1],
              x, 2 * x[-1] - x[-1:-window_len:-1]]
    #print(len(s))
 
    if window == 'flat':  # moving average
        w = np.ones(window_len, 'd')
    else:
        w = getattr(np, window)(window_len)
    y = np.convolve(w / w.sum(), s, mode='same')
    return y[window_len - 1:-window_len + 1]
 

class Frame:
    """class to hold information about each frame
    
    """
    def __init__(self, id, diff):
        self.id = id
        self.diff = diff
 
    def __lt__(self, other):
        if self.id == other.id:
            return self.id < other.id
        return self.id < other.id
 
    def __gt__(self, other):
        return other.__lt__(self)
 
    def __eq__(self, other):
        return self.id == other.id and self.id == other.id
 
    def __ne__(self, other):
        return not self.__eq__(other)
 
 
def rel_change(a, b):
   x = (b - a) / max(a, b)
   print(x)
   return x
 
    
def extract_kvm4(
    USE_THRESH = False,
    THRESH=0.6,
    USE_TOP_ORDER = False,
    NUM_TOP_FRAMES=None,
    USE_LOCAL_MAXIMA=True,
):


    print(sys.executable)
    #Setting fixed threshold criteria
    USE_THRESH = False
    #fixed threshold value
    THRESH = 0.6
    #Setting fixed threshold criteria
    USE_TOP_ORDER = True
    #Number of top sorted frames
    NUM_TOP_FRAMES = 10
    #Setting local maxima criteria
    USE_LOCAL_MAXIMA = False
     
    #Video path of the source file
    videopath = str(path_mp4)
    #Directory to store the processed frames
    dir = 'extract_result/'
    #smoothing window size
    len_window = int(50)
    
    
    print("target video :" + videopath)
    print("frame save directory: " + dir)
    # load video and compute diff between frames
    cap = cv2.VideoCapture(str(videopath)) 
    curr_frame = None
    prev_frame = None 
    frame_diffs = []
    frames = []
    success, frame = cap.read()
    i = 0 
    while(success):
        luv = cv2.cvtColor(frame, cv2.COLOR_BGR2LUV)
        curr_frame = luv
        if curr_frame is not None and prev_frame is not None:
            #logic here
            diff = cv2.absdiff(curr_frame, prev_frame)
            diff_sum = np.sum(diff)
            diff_sum_mean = diff_sum / (diff.shape[0] * diff.shape[1])
            frame_diffs.append(diff_sum_mean)
            frame = Frame(i, diff_sum_mean)
            frames.append(frame)
        prev_frame = curr_frame
        i = i + 1
        success, frame = cap.read()   
    cap.release()
    
    # compute keyframe
    keyframe_id_set = set()
    if USE_TOP_ORDER:
        # sort the list in descending order
        frames.sort(key=operator.attrgetter("diff"), reverse=True)
        for keyframe in frames[:NUM_TOP_FRAMES]:
            keyframe_id_set.add(keyframe.id) 
    if USE_THRESH:
        print("Using Threshold")
        for i in range(1, len(frames)):
            if (rel_change(np.float(frames[i - 1].diff), np.float(frames[i].diff)) >= THRESH):
                keyframe_id_set.add(frames[i].id)   
    if USE_LOCAL_MAXIMA:
        print("Using Local Maxima")
        diff_array = np.array(frame_diffs)
        sm_diff_array = smooth(diff_array, len_window)
        frame_indexes = np.asarray(argrelextrema(sm_diff_array, np.greater))[0]
        for i in frame_indexes:
            keyframe_id_set.add(frames[i - 1].id)
            
        plt.figure(figsize=(40, 20))
        # plt.locator_params(numticks=100)
        plt.stem(sm_diff_array)
        plt.savefig(dir + 'plot.png')
    print(keyframe_id_set)
    # save all keyframes as image
    cap = cv2.VideoCapture(str(videopath))
    curr_frame = None
    keyframes = []
    success, frame = cap.read()
    idx = 0
    while(success):
        if idx in keyframe_id_set:
            name = "keyframe_" + str(idx) + ".jpg"
            cv2.imwrite(dir + name, frame)
            keyframe_id_set.remove(idx)
        idx = idx + 1
        success, frame = cap.read()
    cap.release()


In [ ]:
keyframe_id_set

NameError: name 'keyframe_id_set' is not defined